In [5]:
import logging

from typing import List, Any, Optional, Dict, Tuple
from llama_index.vector_stores.types import (
    VectorStore,
    VectorStoreQuery,
    VectorStoreQueryResult,
)
import llama_index
from llama_index import StorageContext, VectorStoreIndex, ServiceContext
from llama_index.schema import TextNode, BaseNode, Document
from llama_index.memory import ChatMemoryBuffer
from tokenizers import Tokenizer

import oci
import ads

# only to check versions!
import oracledb

from oci_utils import load_oci_config
from oracle_vector_db import OracleVectorStore

# using factory methods
from prepare_chain import create_reranker, create_embedding_model, create_llm
from config import TOKENIZER

In [6]:
# version I'm using
print(f"oracledb version: {oracledb.__version__}")
print(f"oci version: {oci.__version__}")
print(f"llama_index version: {llama_index.__version__}")

oracledb version: 2.0.0.dev20231121
oci version: 2.112.1+preview.1.1649
llama_index version: 0.9.21


In [7]:
cohere_tokenizer = Tokenizer.from_pretrained(TOKENIZER)

In [8]:
# parameters

# for similarity query
TOP_K = 4
# for Reranker
TOP_N = 2

In [10]:
# Build the entire chain
oci_config = load_oci_config()

# need to do this way
api_keys_config = ads.auth.api_keys(oci_config)

# english, or for other language use: multilingual

# Embedding model
embed_model = create_embedding_model(
    auth=api_keys_config
)

# Vector Store
v_store = OracleVectorStore(verbose=False)

# reranker
# Reranker is deployed in OCI Data Science as a Model Deployment

reranker = create_reranker(auth=api_keys_config, verbose=True)

# LLM
llm_oci = create_llm(auth=api_keys_config)

# Llama-index
service_context = ServiceContext.from_defaults(llm=llm_oci, embed_model=embed_model)

index = VectorStoreIndex.from_vector_store(
    vector_store=v_store, service_context=service_context
)

2024-01-04 18:33:06,278 - INFO - Created OCI reranker client...
2024-01-04 18:33:06,279 - INFO - Region: eu-frankfurt-1...
2024-01-04 18:33:06,279 - INFO - Deployment id: ocid1.datasciencemodeldeployment.oc1.eu-frankfurt-1.amaaaaaangencdyaulxbosgii6yajt2jdsrrvfbequkxt3mepz675uk3ui3q...
2024-01-04 18:33:06,279 - INFO - 


#### To add message_history, create a chat_engine

In [11]:
memory = ChatMemoryBuffer.from_defaults(
    token_limit=2800, tokenizer_fn=cohere_tokenizer.encode
)

chat_engine = index.as_chat_engine(
    chat_mode="context",
    memory=memory,
    verbose=True,
    similarity_top_k=TOP_K,
    node_postprocessors=[reranker],
)

In [12]:
# start a new conversation
chat_engine.reset()

In [13]:
memory.chat_history

[]

In [14]:
response = chat_engine.chat("What is the last release for Oracle database?")

print(response.response)

2024-01-04 18:33:13,149 - INFO - Reranking...
2024-01-04 18:33:13,484 - INFO - ...elapsed time: 0.34 sec.


 The last release for Oracle Database that I could find is Oracle Database Release 23c dated October 2023. 

Would you like to know more about this release?  I can list some new features that it includes if you'd like. 
Alternatively, I can search for more recent updates or releases if you provide more information or direct me to a specific source for my search. 


In [15]:
memory.chat_history

[ChatMessage(role=<MessageRole.USER: 'user'>, content='What is the last release for Oracle database?', additional_kwargs={}),
 ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content=" The last release for Oracle Database that I could find is Oracle Database Release 23c dated October 2023. \n\nWould you like to know more about this release?  I can list some new features that it includes if you'd like. \nAlternatively, I can search for more recent updates or releases if you provide more information or direct me to a specific source for my search. ", additional_kwargs={})]

In [16]:
response = chat_engine.chat("Is it a long term release?")

print(response.response)

2024-01-04 18:33:18,937 - INFO - Reranking...
2024-01-04 18:33:19,170 - INFO - ...elapsed time: 0.23 sec.


 Yes, Oracle Database Release 23c is a long-term support release.

Oracle Database Release 19c was the first long-term support release and it was released in December 2019. Oracle Database Release 21c was made available in April 2021 and Release 22c was released in September 2022. Now, Release 23c has been released in October 2023 and it will be fully supported for an extended period of time, offering customers a stable and reliable platform for their databases. 

Would you like to know more about the long-term release strategy of Oracle Database? I can provide more details on the benefits or implications of choosing a long-term release compared to more recent releases. 


In [17]:
memory.chat_history

[ChatMessage(role=<MessageRole.USER: 'user'>, content='What is the last release for Oracle database?', additional_kwargs={}),
 ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content=" The last release for Oracle Database that I could find is Oracle Database Release 23c dated October 2023. \n\nWould you like to know more about this release?  I can list some new features that it includes if you'd like. \nAlternatively, I can search for more recent updates or releases if you provide more information or direct me to a specific source for my search. ", additional_kwargs={}),
 ChatMessage(role=<MessageRole.USER: 'user'>, content='Is it a long term release?', additional_kwargs={}),
 ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content=' Yes, Oracle Database Release 23c is a long-term support release.\n\nOracle Database Release 19c was the first long-term support release and it was released in December 2019. Oracle Database Release 21c was made available in April 2021 and Rel

In [18]:
response = chat_engine.chat("What about new features for JSON?")

print(response.response)

2024-01-04 18:33:39,699 - INFO - Reranking...
2024-01-04 18:33:40,385 - INFO - ...elapsed time: 0.69 sec.


 Certainly! Oracle Database Release 23c includes several new features and enhancements for JSON support. Here are some of the key improvements:

1. **JSON-Relational Duality Views**: Customers can now create fully updatable JSON views over relational data. This feature provides flexibility by allowing data to be accessed in the form of JSON documents while still being stored in highly efficient relational tables.

2. **JSON Schema Validation**: JSON Schema-based validation is now supported. Customers can define a JSON schema document that specifies the allowed properties and data types for their JSON data, ensuring consistent structures and data typing.

3. **Consistent XML and JSON Search Index Syntax**: The syntax for creating and using search indexes on XML and JSON data is now consistent, which simplifies productivity for developers working with both data formats. Additionally, the performance of JSON and XML search indexes has been improved.

4. **Native Database Support for JSON*

In [ ]:
memory.chat_history

In [ ]:
response = chat_engine.chat("Yes give me some more details regarding the point n. 1")

print(response.response)

In [ ]:
memory.chat_history

In [ ]:
response = chat_engine.chat("What about the symptoms for long covid?")

print(response.response)

In [ ]:
memory.chat_history

In [ ]:
response = chat_engine.chat("Tell me more about Respiratory issues")

print(response.response)

In [ ]:
memory.chat_history